In [58]:
import tensorflow as tf
import numpy as np

In [59]:
mnist = tf.keras.datasets.mnist

In [60]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0 # Normalize values between 0 and 1

In [61]:
y_train[0:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [62]:
print("Categorical Labels")
print(y_train[0:5])

y_train = tf.one_hot(y_train, 10)
y_test = tf.one_hot(y_test, 10)

print("One hot encoded labels")
print(y_train[0:5])

Categorical Labels
[5 0 4 1 9]
One hot encoded labels
tf.Tensor(
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]], shape=(5, 10), dtype=float32)


In [63]:
print("Number of training examples : ", X_train.shape[0])
print("Number of test examples", X_test.shape[0] )

Number of training examples :  60000
Number of test examples 10000


In [64]:
# size of image in pixels
width = 28
height = 28

# size of flat array
flat = width * height
# number of output classifications
class_output = 10


In [65]:
X_image_train = tf.reshape(X_train, [-1,28,28,1])  # [batch_dim, ... input_shape] 28x28x1 === width x height x color (greyscale)
X_image_train = tf.cast(X_image_train, 'float32')

X_image_test = tf.reshape(X_test, [-1,28,28,1])
X_image_test = tf.cast(X_image_test, 'float32')

In [66]:
train_ds = tf.data.Dataset.from_tensor_slices((X_image_train, y_train)).batch(50)
test_ds = tf.data.Dataset.from_tensor_slices((X_image_test, y_test)).batch(50)

# Convolutional Layer 1

In [67]:
W_conv1 = tf.Variable(tf.random.truncated_normal([5, 5, 1, 32], stddev = 0.1, seed = 0))
b_conv_1 = tf.Variable(tf.constant(0.1, shape=[32]))

In [68]:
def convolve1(x):
    return (
        tf.nn.conv2d(x,W_conv1, strides = [1,1,1,1], padding = 'SAME') + b_conv_1
    )

In [69]:
def h_conv1(x):
    return (
        tf.nn.relu(convolve1(x))
    )

In [70]:
def conv1(x):
    return (
        tf.nn.max_pool(h_conv1(x), ksize= [1,2,2,1], strides = [1,2,2,1], padding='SAME')
    )

# Convolutional Layer 2

In [71]:
W_conv2 = tf.Variable(tf.random.truncated_normal([5,5,32,64], stddev=0.1, seed = 1))
b_conv2 = tf.constant(tf.constant(0.1, shape = [64]))

In [72]:
def convolve2(x):
    return (
        tf.nn.conv2d(conv1(x), W_conv2, strides=[1,1,1,1], padding='SAME') + b_conv2
    )

In [73]:
def h_conv2(x):
    return (
        tf.nn.relu(convolve2(x))
    )

In [74]:
def conv2(x):
    return (
        tf.nn.max_pool(h_conv2(x), ksize = [1,2,2,1], strides=[1,2,2,1], padding = 'SAME')
    )

# Fully Connected Layer

In [75]:
def layer2_matrix(x):
    return tf.reshape(conv2(x), [-1, 7*7*64])

In [76]:
W_fc1 = tf.Variable(tf.random.truncated_normal([7*7*64,1024], stddev=0.1, seed = 2))
b_fc1 = tf.constant(0.1, shape=[1024])

In [77]:
def fcl(x):
    return tf.matmul(layer2_matrix(x), W_fc1) + b_fc1

In [78]:
def h_fc1(x):
    return tf.nn.relu(fcl(x))

# Dropout Layer

In [79]:
keep_prob = 0.5
def layer_drop(x):
    return tf.nn.dropout(h_fc1(x), keep_prob)

# SoftMax Layer

In [80]:
W_fc2 = tf.Variable(tf.random.truncated_normal([1024, 10], stddev=0.1, seed = 2))
b_fc2 = tf.Variable(tf.constant(0.1, shape = [10]))

In [81]:
def fc(x):
    return tf.matmul(layer_drop(x), W_fc2) + b_fc2

In [82]:
def y_CNN(x):
    return tf.nn.softmax(fc(x))

# Loss Function : Cross Entropy

In [83]:
def cross_entropy(y_label, y_pred):
    return (-tf.reduce_sum(y_label * tf.math.log(y_pred + 1.e-10)))

In [84]:
optimizer = tf.keras.optimizers.Adam(1e-4)

In [85]:
variables = [W_conv1, b_conv_1, W_conv2, b_conv2, W_fc1, W_fc2, b_fc2]

In [86]:
def train_step(x,y):
    with tf.GradientTape() as tape:
        current_loss = cross_entropy(y, y_CNN(x))
        grads = tape.gradient(current_loss, variables)
        optimizer.apply_gradients(zip(grads, variables))
        return current_loss.numpy()

In [87]:
correct_prediction = tf.equal(tf.argmax(y_CNN(X_image_train), axis=1), tf.argmax(y_train, axis=1))

In [88]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float32'))

In [91]:
print(correct_prediction)

tf.Tensor([False  True False ... False False False], shape=(60000,), dtype=bool)


In [89]:
loss_values=[]
accuracies = []
epochs = 1

for i in range(epochs):
    j=0
    # each batch has 50 examples
    for x_train_batch, y_train_batch in train_ds:
        j+=1
        current_loss = train_step(x_train_batch, y_train_batch)
        if j%50==0: #reporting intermittent batch statistics
            correct_prediction = tf.equal(tf.argmax(y_CNN(x_train_batch), axis=1),
                                  tf.argmax(y_train_batch, axis=1))
            #  accuracy
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)).numpy()
            print("epoch ", str(i), "batch", str(j), "loss:", str(current_loss),
                     "accuracy", str(accuracy)) 
            
    current_loss = cross_entropy( y_train, y_CNN( X_image_train )).numpy()
    loss_values.append(current_loss)
    correct_prediction = tf.equal(tf.argmax(y_CNN(X_image_train), axis=1),
                                  tf.argmax(y_train, axis=1))
    #  accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)).numpy()
    accuracies.append(accuracy)
    print("end of epoch ", str(i), "loss", str(current_loss), "accuracy", str(accuracy) )  
            

AttributeError: Tensor.name is meaningless when eager execution is enabled.